## Hail Calculation with simple approach

### 💧 Mass–Diameter Relationship for Hydrometeors

This method is based on a simple **mass–diameter relationship** using:

$$
m = \frac{q}{n}
$$

> where:
> $m$ = average mass per particle \[kg]
> $q$ = mass mixing ratio \[kg/kg]
> $n$ = number concentration \[#/kg]

---

Assuming **spherical particles**, the volume–mass relation is:

$$
m = \frac{\pi}{6} \rho_p D^3 \quad \Rightarrow \quad D = \left( \frac{6m}{\pi \rho_p} \right)^{1/3}
$$

Where:

* $\rho_p$ = particle material density \[kg/m³]
* $D$ = diameter \[m]

---

### 📌 Suggested Hydrometeor Densities

Use realistic values for $\rho_p$ depending on the particle type:

* **Rain**: \~1000 kg/m³
* **Graupel**: \~400–500 kg/m³
* **Hail**: \~700–900 kg/m³

---

### ✅ Final Formula (Diameter in meters)

$$
D = \left( \frac{6 \cdot \frac{q}{n}}{\pi \cdot \rho_p} \right)^{1/3}
$$

Or simplified:

$$
D = \left( \frac{6q}{n \pi \rho_p} \right)^{1/3}
$$

Where you define $\rho_p$ as appropriate for each hydrometeor type.

---

Let me know if you also want a version rendered for PDF or presentation slides.


In [1]:
import os
import xarray as xr
import numpy as np
from tqdm import tqdm

# === Function to calculate diameter ===
def compute_d(q, n, rho_p):
    with np.errstate(divide='ignore', invalid='ignore'):
        m = xr.where(n > 0, q / n, np.nan)
        D = ((6 * m) / (np.pi * rho_p))**(1/3)
    return D

def calculate_hydrometeor_diameters(ds):
    # Prescribed hydrometeor densities [kg/m³]
    rho_rain = 1000.0
    rho_graupel = 500.0
    rho_hail = 900.0

    qr = ds["qr"]
    nr = ds["nr"]
    qg = ds["qg"]
    ng = ds["ng"]
    qh = ds["qh"]
    nh = ds["nh"]

    time_dim = ds.sizes["time"]
    diam_rain = []
    diam_graupel = []
    diam_hail = []

    for t in tqdm(range(time_dim), desc="Calculating diameters over time"):
        d_rain = compute_d(qr.isel(time=t), nr.isel(time=t), rho_rain)
        d_grau = compute_d(qg.isel(time=t), ng.isel(time=t), rho_graupel)
        d_hail = compute_d(qh.isel(time=t), nh.isel(time=t), rho_hail)

        diam_rain.append(d_rain)
        diam_graupel.append(d_grau)
        diam_hail.append(d_hail)

    # Stack back into DataArray with time
    rain_d_da = xr.concat(diam_rain, dim="time")
    graupel_d_da = xr.concat(diam_graupel, dim="time")
    hail_d_da = xr.concat(diam_hail, dim="time")

    rain_d_da.name = "diameter_raindrop"
    graupel_d_da.name = "diameter_graupel"
    hail_d_da.name = "diameter_hail"

    rain_d_da.attrs = {'units': 'm', 'long_name': 'Raindrop diameter'}
    graupel_d_da.attrs = {'units': 'm', 'long_name': 'Graupel diameter'}
    hail_d_da.attrs = {'units': 'm', 'long_name': 'Hail diameter'}

    return rain_d_da, graupel_d_da, hail_d_da


In [2]:
# === Input/Output Setup ===
input_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"
output_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"

selected_sets = [2, 12, 11, 8, 9, 10, 5]
file_template = "filtered_w10_new_set{}.nc"

for i in selected_sets:
    path_in = os.path.join(input_dir, file_template.format(i))
    path_out = os.path.join(output_dir, f"filtered_w10_new_dia_set{i}.nc")

    if os.path.exists(path_in):
        print(f"\nProcessing set {i}...")
        ds = xr.open_dataset(path_in)

        rain_d, graupel_d, hail_d = calculate_hydrometeor_diameters(ds)

        # Merge and save
        ds_out = ds.copy()
        ds_out["diameter_raindrop"] = rain_d
        ds_out["diameter_graupel"] = graupel_d
        ds_out["diameter_hail"] = hail_d

        ds_out.to_netcdf(path_out)
        print(f"Saved: {path_out}")
    else:
        print(f"File not found: {path_in}")


Processing set 2...


Calculating diameters over time: 100%|██████████| 121/121 [11:50<00:00,  5.88s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w10_new_dia_set2.nc

Processing set 12...


Calculating diameters over time: 100%|██████████| 121/121 [12:18<00:00,  6.10s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w10_new_dia_set12.nc

Processing set 11...


...

Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w10_new_dia_set9.nc

Processing set 10...


Calculating diameters over time: 100%|██████████| 121/121 [12:04<00:00,  5.99s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w10_new_dia_set10.nc

Processing set 5...


Calculating diameters over time: 100%|██████████| 121/121 [12:16<00:00,  6.08s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w10_new_dia_set5.nc


In [ ]:
import os
import xarray as xr
import numpy as np

# === Input Directory and File Pattern ===
input_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"
selected_sets = [2, 12, 11, 8, 9, 10, 5]
file_template = "filtered_w10_new_dia_set{}.nc"

# === Variable Conversion Mapping (name, units, scale factor) ===
variables = {
    "diameter_raindrop": {"label": "Raindrop", "unit": "µm", "scale": 1e6},
    "diameter_graupel": {"label": "Graupel", "unit": "mm", "scale": 1e3},
    "diameter_hail": {"label": "Hail", "unit": "mm", "scale": 1e3},
}

# === Store results grouped by variable
stats_by_var = {var: {} for var in variables}

# === Loop over datasets
for i in selected_sets:
    file_path = os.path.join(input_dir, file_template.format(i))
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue

    ds = xr.open_dataset(file_path)

    for varname, props in variables.items():
        if varname not in ds:
            print(f"Variable {varname} not found in set {i}")
            continue

        data = ds[varname] * props["scale"]
        vals = data.values.flatten()
        vals = vals[~np.isnan(vals)]

        # Compute statistics
        stats = {
            "max": np.nanmax(vals),
            "p99": np.nanpercentile(vals, 99),
            "p95": np.nanpercentile(vals, 95),
            "median": np.nanmedian(vals),
            "mean": np.nanmean(vals),
        }

        stats_by_var[varname][f"set{i}"] = stats

    ds.close()

# === Print grouped by variable
for varname, set_stats in stats_by_var.items():
    label = variables[varname]["label"]
    unit = variables[varname]["unit"]
    print(f"\n=== {label} Diameter Stats [{unit}] ===")
    print(f"{'Dataset':<8}  {'Max':>8}  {'99th':>8}  {'95th':>8}  {'Median':>8}  {'Mean':>8}")
    print("-" * 55)
    for dataset, stats in set_stats.items():
        print(f"{dataset:<8}  {stats['max']:8.2f}  {stats['p99']:8.2f}  {stats['p95']:8.2f}  "
              f"{stats['median']:8.2f}  {stats['mean']:8.2f}")


#### Raindrop Size Diameter

In [1]:
import os
import xarray as xr
import numpy as np
from tqdm import tqdm

# === Function to calculate diameter ===
def compute_dr(q, n, rho_p):
    with np.errstate(divide='ignore', invalid='ignore'):
        m = xr.where(n > 0, q / n, np.nan)
        D = ((6 * m) / (np.pi * rho_p))**(1/3)
    return D

def calculate_hydrometeor_diameter_rain(ds):
    # Prescribed hydrometeor densities [kg/m³]
    rho_rain = 1000.0

    qr = ds["qr"]
    nr = ds["nr"]

    time_dim = ds.sizes["time"]
    diam_rain = []

    for t in tqdm(range(time_dim), desc="Calculating diameters over time"):
        d_rain = compute_dr(qr.isel(time=t), nr.isel(time=t), rho_rain)

        diam_rain.append(d_rain)

    # Stack back into DataArray with time
    rain_d_da = xr.concat(diam_rain, dim="time")
    rain_d_da.name = "diameter_raindrop"
    rain_d_da.attrs = {'units': 'm', 'long_name': 'Raindrop diameter'}
    
    return rain_d_da


In [5]:
# === Input/Output Setup ===
input_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"
output_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"

selected_sets = [1, 2, 3, 4, 5, 6, 7]
file_template = "filtered_w5_hres_set{}.nc"

for i in selected_sets:
    path_in = os.path.join(input_dir, file_template.format(i))
    path_out = os.path.join(output_dir, f"filtered_w5_hres_new_dia_set{i}.nc")

    if os.path.exists(path_in):
        print(f"\nProcessing set {i}...")
        ds = xr.open_dataset(path_in)

        rain_d = calculate_hydrometeor_diameter_rain(ds)

        # Merge and save
        ds_out = ds.copy()
        ds_out["diameter_raindrop"] = rain_d
        
        ds_out.to_netcdf(path_out)
        print(f"Saved: {path_out}")
    else:
        print(f"File not found: {path_in}")


Processing set 1...


Calculating diameters over time: 100%|██████████| 101/101 [03:33<00:00,  2.11s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hres_new_dia_set1.nc

Processing set 2...


Calculating diameters over time: 100%|██████████| 101/101 [03:31<00:00,  2.10s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hres_new_dia_set2.nc

Processing set 3...


...

Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hres_new_dia_set5.nc

Processing set 6...


Calculating diameters over time: 100%|██████████| 101/101 [03:21<00:00,  1.99s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hres_new_dia_set6.nc

Processing set 7...


Calculating diameters over time: 100%|██████████| 101/101 [03:22<00:00,  2.01s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hres_new_dia_set7.nc


In [2]:
import os
import xarray as xr
import numpy as np

# === Input Directory and File Pattern ===
input_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"
selected_sets = [1, 2, 3, 4, 5, 6, 7]
file_template = "filtered_w5_hres_new_dia_set{}.nc"

# === Variable Conversion Mapping (name, units, scale factor) ===
variables = {
    "diameter_raindrop": {"label": "Raindrop", "unit": "µm", "scale": 1e6},
}

# === Store results grouped by variable
stats_by_var = {var: {} for var in variables}

# === Loop over datasets
for i in selected_sets:
    file_path = os.path.join(input_dir, file_template.format(i))
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue

    ds = xr.open_dataset(file_path)

    for varname, props in variables.items():
        if varname not in ds:
            print(f"Variable {varname} not found in set {i}")
            continue

        data = ds[varname] * props["scale"]
        vals = data.values.flatten()
        vals = vals[~np.isnan(vals)]

        # Compute statistics
        stats = {
            "max": np.nanmax(vals),
            "p99": np.nanpercentile(vals, 99),
            "p95": np.nanpercentile(vals, 95),
            "median": np.nanmedian(vals),
            "mean": np.nanmean(vals),
        }

        stats_by_var[varname][f"set{i}"] = stats

    ds.close()

# === Print grouped by variable
for varname, set_stats in stats_by_var.items():
    label = variables[varname]["label"]
    unit = variables[varname]["unit"]
    print(f"\n=== {label} Diameter Stats [{unit}] ===")
    print(f"{'Dataset':<8}  {'Max':>8}  {'99th':>8}  {'95th':>8}  {'Median':>8}  {'Mean':>8}")
    print("-" * 55)
    for dataset, stats in set_stats.items():
        print(f"{dataset:<8}  {stats['max']:8.2f}  {stats['p99']:8.2f}  {stats['p95']:8.2f}  "
              f"{stats['median']:8.2f}  {stats['mean']:8.2f}")



=== Raindrop Diameter Stats [µm] ===
Dataset        Max      99th      95th    Median      Mean
-------------------------------------------------------
set1        775.70    328.96    175.87     98.18    105.76
set2        745.38    271.13    126.46     94.54    100.35
set3        787.39    263.13    138.42     95.76    100.96
set4        630.69    222.27    123.16     94.44     98.80
set5        409.40    201.87    118.53     93.39     97.41
set6        516.55    203.27    115.17     92.03     96.62
set7        625.46    250.93    128.36     92.41     98.72


In [3]:
# === Input/Output Setup ===
input_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"
output_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"

selected_sets = [6, 7]
file_template = "filtered_w3_hres_set{}.nc"

for i in selected_sets:
    path_in = os.path.join(input_dir, file_template.format(i))
    path_out = os.path.join(output_dir, f"filtered_w3_hres_new_dia_set{i}.nc")

    if os.path.exists(path_in):
        print(f"\nProcessing set {i}...")
        ds = xr.open_dataset(path_in)

        rain_d = calculate_hydrometeor_diameter_rain(ds)

        # Merge and save
        ds_out = ds.copy()
        ds_out["diameter_raindrop"] = rain_d
        
        ds_out.to_netcdf(path_out)
        print(f"Saved: {path_out}")
    else:
        print(f"File not found: {path_in}")


Processing set 6...


Calculating diameters over time: 100%|██████████| 101/101 [03:31<00:00,  2.09s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w3_hres_new_dia_set6.nc

Processing set 7...


Calculating diameters over time: 100%|██████████| 101/101 [04:01<00:00,  2.39s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w3_hres_new_dia_set7.nc


In [4]:
import os
import xarray as xr
import numpy as np

# === Input Directory and File Pattern ===
input_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"
selected_sets = [1, 2, 3, 4, 5, 6, 7]
file_template = "filtered_w3_hres_new_dia_set{}.nc"

# === Variable Conversion Mapping (name, units, scale factor) ===
variables = {
    "diameter_raindrop": {"label": "Raindrop", "unit": "µm", "scale": 1e6},
}

# === Store results grouped by variable
stats_by_var = {var: {} for var in variables}

# === Loop over datasets
for i in selected_sets:
    file_path = os.path.join(input_dir, file_template.format(i))
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue

    ds = xr.open_dataset(file_path)

    for varname, props in variables.items():
        if varname not in ds:
            print(f"Variable {varname} not found in set {i}")
            continue

        data = ds[varname] * props["scale"]
        vals = data.values.flatten()
        vals = vals[~np.isnan(vals)]

        # Compute statistics
        stats = {
            "max": np.nanmax(vals),
            "p99": np.nanpercentile(vals, 99),
            "p95": np.nanpercentile(vals, 95),
            "median": np.nanmedian(vals),
            "mean": np.nanmean(vals),
        }

        stats_by_var[varname][f"set{i}"] = stats

    ds.close()

# === Print grouped by variable
for varname, set_stats in stats_by_var.items():
    label = variables[varname]["label"]
    unit = variables[varname]["unit"]
    print(f"\n=== {label} Diameter Stats [{unit}] ===")
    print(f"{'Dataset':<8}  {'Max':>8}  {'99th':>8}  {'95th':>8}  {'Median':>8}  {'Mean':>8}")
    print("-" * 55)
    for dataset, stats in set_stats.items():
        print(f"{dataset:<8}  {stats['max']:8.2f}  {stats['p99']:8.2f}  {stats['p95']:8.2f}  "
              f"{stats['median']:8.2f}  {stats['mean']:8.2f}")



=== Raindrop Diameter Stats [µm] ===
Dataset        Max      99th      95th    Median      Mean
-------------------------------------------------------
set1        775.70    313.00    166.91     97.13    104.34
set2        745.38    256.02    126.78     94.87     99.87
set3        787.39    256.93    138.04     95.61    100.61
set4        630.69    223.13    125.37     94.68     98.77
set5        418.57    214.42    123.24     93.91     97.99
set6        516.55    221.57    120.96     92.87     97.54
set7        625.46    231.46    117.42     91.78     96.91


In [1]:
import os
import xarray as xr
import numpy as np
from tqdm import tqdm

# === Function to calculate diameter ===
def compute_d(q, n, rho_p):
    with np.errstate(divide='ignore', invalid='ignore'):
        m = xr.where(n > 0, q / n, np.nan)
        D = ((6 * m) / (np.pi * rho_p))**(1/3)
    return D

def calculate_hydrometeor_diameter_hg(ds):
    # Prescribed hydrometeor densities [kg/m³]
    rho_graupel = 500.0
    rho_hail = 900.0
    
    qg = ds["qg"]
    ng = ds["ng"]
    qh = ds["qh"]
    nh = ds["nh"]

    time_dim = ds.sizes["time"]
    diam_graupel = []
    diam_hail = []

    for t in tqdm(range(time_dim), desc="Calculating diameters over time"):
        d_grau = compute_d(qg.isel(time=t), ng.isel(time=t), rho_graupel)
        d_hail = compute_d(qh.isel(time=t), nh.isel(time=t), rho_hail)

        diam_graupel.append(d_grau)
        diam_hail.append(d_hail)

    # Stack back into DataArray with time
    graupel_d_da = xr.concat(diam_graupel, dim="time")
    hail_d_da = xr.concat(diam_hail, dim="time")

    graupel_d_da.name = "diameter_graupel"
    hail_d_da.name = "diameter_hail"

    graupel_d_da.attrs = {'units': 'm', 'long_name': 'Graupel diameter'}
    hail_d_da.attrs = {'units': 'm', 'long_name': 'Hail diameter'}

    return graupel_d_da, hail_d_da


In [2]:
# === Input/Output Setup ===
input_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"
output_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"

selected_sets = [1, 2, 3, 4, 5]
file_template = "filtered_w5_hg_new_set{}.nc"

for i in selected_sets:
    path_in = os.path.join(input_dir, file_template.format(i))
    path_out = os.path.join(output_dir, f"filtered_w5_hg_new_dia_set{i}.nc")

    if os.path.exists(path_in):
        print(f"\nProcessing set {i}...")
        ds = xr.open_dataset(path_in)

        graupel_d, hail_d = calculate_hydrometeor_diameter_hg(ds)

        # Merge and save
        ds_out = ds.copy()
        ds_out["diameter_graupel"] = graupel_d
        ds_out["diameter_hail"] = hail_d

        ds_out.to_netcdf(path_out)
        print(f"Saved: {path_out}")
    else:
        print(f"File not found: {path_in}")


Processing set 1...


Calculating diameters over time: 100%|██████████| 101/101 [15:35<00:00,  9.26s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hg_new_dia_set1.nc

Processing set 2...


Calculating diameters over time: 100%|██████████| 101/101 [26:43<00:00, 15.88s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hg_new_dia_set2.nc

Processing set 3...


...

Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hg_new_dia_set3.nc

Processing set 4...


Calculating diameters over time: 100%|██████████| 101/101 [22:59<00:00, 13.66s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hg_new_dia_set4.nc

Processing set 5...


Calculating diameters over time: 100%|██████████| 101/101 [18:06<00:00, 10.76s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hg_new_dia_set5.nc


In [3]:
# === Input/Output Setup ===
input_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"
output_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"

selected_sets = [6, 7]
file_template = "filtered_w5_hg_new_set{}.nc"

for i in selected_sets:
    path_in = os.path.join(input_dir, file_template.format(i))
    path_out = os.path.join(output_dir, f"filtered_w5_hg_new_dia_set{i}.nc")

    if os.path.exists(path_in):
        print(f"\nProcessing set {i}...")
        ds = xr.open_dataset(path_in)

        graupel_d, hail_d = calculate_hydrometeor_diameter_hg(ds)

        # Merge and save
        ds_out = ds.copy()
        ds_out["diameter_graupel"] = graupel_d
        ds_out["diameter_hail"] = hail_d

        ds_out.to_netcdf(path_out)
        print(f"Saved: {path_out}")
    else:
        print(f"File not found: {path_in}")


Processing set 6...


Calculating diameters over time: 100%|██████████| 101/101 [17:49<00:00, 10.59s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hg_new_dia_set6.nc

Processing set 7...


Calculating diameters over time: 100%|██████████| 101/101 [17:22<00:00, 10.32s/it]


Saved: /glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data/filtered_w5_hg_new_dia_set7.nc


In [5]:
import os
import xarray as xr
import numpy as np

# === Input Directory and File Pattern ===
input_dir = "/glade/derecho/scratch/fsari/CM1/DATA/new_ramp/bismillah_final/filtered_data"
selected_sets = [1, 2, 3, 4, 5, 6, 7]
file_template = "filtered_w5_hg_new_dia_set{}.nc"

# === Variable Conversion Mapping (name, units, scale factor) ===
variables = {
    "diameter_graupel": {"label": "Graupel", "unit": "mm", "scale": 1e3},
    "diameter_hail": {"label": "Hail", "unit": "mm", "scale": 1e3},
}

# === Store results grouped by variable
stats_by_var = {var: {} for var in variables}

# === Loop over datasets
for i in selected_sets:
    file_path = os.path.join(input_dir, file_template.format(i))
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue

    ds = xr.open_dataset(file_path)

    for varname, props in variables.items():
        if varname not in ds:
            print(f"Variable {varname} not found in set {i}")
            continue

        data = ds[varname] * props["scale"]
        vals = data.values.flatten()
        vals = vals[~np.isnan(vals)]

        # Compute statistics
        stats = {
            "max": np.nanmax(vals),
            "p99": np.nanpercentile(vals, 99),
            "p95": np.nanpercentile(vals, 95),
            "median": np.nanmedian(vals),
            "mean": np.nanmean(vals),
           }
        stats_by_var[varname][f"set{i}"] = stats

    ds.close()

# === Print grouped by variable
for varname, set_stats in stats_by_var.items():
    label = variables[varname]["label"]
    unit = variables[varname]["unit"]
    print(f"\n=== {label} Diameter Stats [{unit}] ===")
    print(f"{'Dataset':<8}  {'Max':>8}  {'99th':>8}  {'95th':>8}  {'Median':>8}  {'Mean':>8}")
    print("-" * 55)
    for dataset, stats in set_stats.items():
        print(f"{dataset:<8}  {stats['max']:8.2f}  {stats['p99']:8.2f}  {stats['p95']:8.2f}  "
              f"{stats['median']:8.2f}  {stats['mean']:8.2f}")


ValueError: zero-size array to reduction operation fmax which has no identity